In [1]:
import pandas as pd
import polars
from nba_api.stats import endpoints as nba_endpoints
from nba_api.live.nba import endpoints as nba_live_endpoints
import plotly.graph_objects as go
import multiprocessing as mp
from tqdm.notebook import tqdm
import asyncio
import nest_asyncio
from parallel_pandas import ParallelPandas
nest_asyncio.apply()

In [2]:
pd.set_option("display.max_rows", 100)

# GW or Tying Shots Analysis

In [3]:
playerindex = nba_endpoints.playerindex.PlayerIndex()
data = playerindex.data_sets[0].get_dict().get("PlayerIndex").get("data")
headers = playerindex.data_sets[0].get_dict().get("PlayerIndex").get("headers")
players = pd.DataFrame(
    data, columns=headers
)

In [44]:
nba_endpoints.playercareerstats.PlayerCareerStats("1628369").get_data_frames()[0].transpose()

,0,1,2,3,4,5,6
PLAYER_ID,1628369,1628369,1628369,1628369,1628369,1628369,1628369
SEASON_ID,2017-18,2018-19,2019-20,2020-21,2021-22,2022-23,2023-24
LEAGUE_ID,00,00,00,00,00,00,00
TEAM_ID,1610612738,1610612738,1610612738,1610612738,1610612738,1610612738,1610612738
TEAM_ABBREVIATION,BOS,BOS,BOS,BOS,BOS,BOS,BOS
PLAYER_AGE,20.0,21.0,22.0,23.0,24.0,25.0,25.0
GP,80,79,66,64,76,74,38
GS,80,79,66,64,76,74,38
MIN,2443.0,2455.0,2265.0,2290.0,2731.0,2732.0,1360.0
FGM,397,466,552,605,708,727,348


In [8]:
start_year = 2018

In [9]:
seasons = [
    f"{x}-{(x+1)-2000}" for x in range(int(start_year),2024)
]
print(seasons)
# ['2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24']

['2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24']


In [10]:
game = nba_live_endpoints.playbyplay

In [11]:
box = nba_endpoints.boxscoresummaryv2.BoxScoreSummaryV2("0022300421").get_data_frames()[0][
    ["GAME_ID", "HOME_TEAM_ID", "VISITOR_TEAM_ID"]
]

In [12]:
players = pd.DataFrame()
for s in seasons:
    data = playerindex.data_sets[0].get_dict().get("PlayerIndex").get("data")
    headers = playerindex.data_sets[0].get_dict().get("PlayerIndex").get("headers")
    players = pd.concat([players,
        pd.DataFrame(
            data, columns=headers
        ).assign(season=s)[
            ["PERSON_ID", "PLAYER_FIRST_NAME", "PLAYER_LAST_NAME", "TEAM_ID", "TEAM_ABBREVIATION", "season"]
        ]
    ], axis=0)

In [13]:
teamlogs = nba_endpoints.teamgamelogs

game_logs = pd.DataFrame()
for s in seasons:
    game_log = teamlogs.TeamGameLogs(
        season_nullable=s
    ).get_data_frames()[0][["GAME_ID", "MATCHUP"]].assign(
        first_team=lambda row: row["MATCHUP"].str.extract("(^[A-Z]+)"),
        second_team=lambda row: row["MATCHUP"].str.extract("([A-Z]+$)"),
        home_away=lambda row: row["MATCHUP"].str.extract("(@)"),
        home=lambda row: np.where(row["home_away"] == "@", row["second_team"], row["first_team"]),
        away=lambda row: np.where(row["home_away"] == "@", row["first_team"], row["second_team"]),
        season=s
    ).drop(["first_team", "second_team", "home_away"], axis=1)

    game_logs = pd.concat([game_logs, game_log], axis=0)

In [14]:
teams = teamlogs.TeamGameLogs(season_nullable=s).get_data_frames()[0]["TEAM_ID"].unique()

team_info = pd.DataFrame()
for t in teams:
    info = nba_endpoints.teaminfocommon.TeamInfoCommon(team_id=t).get_data_frames()[0][["TEAM_ID", "TEAM_ABBREVIATION"]]
    team_info = pd.concat([team_info, info], axis=0)

In [15]:
game_logs = game_logs.merge(
    team_info,
    left_on="home",
    right_on="TEAM_ABBREVIATION"
).rename(
    columns={"TEAM_ID": "HOME_ID", "TEAM_ABBREVIATION": "HOME_ABBREVIATION"}
).merge(
    team_info,
    left_on="away",
    right_on="TEAM_ABBREVIATION"
).rename(
    columns={"TEAM_ID": "AWAY_ID", "TEAM_ABBREVIATION": "AWAY_ABBREVIATION"}
).drop(["MATCHUP", "home", "away"], axis=1).drop_duplicates().sort_values("GAME_ID")

In [16]:
game_df = teamlogs.TeamGameLogs(season_nullable="2023-24").get_data_frames()[0]

In [17]:
game_df.head().transpose()

,0,1,2,3,4
SEASON_YEAR,2023-24,2023-24,2023-24,2023-24,2023-24
TEAM_ID,1610612741,1610612758,1610612766,1610612765,1610612738
TEAM_ABBREVIATION,CHI,SAC,CHA,DET,BOS
TEAM_NAME,Chicago Bulls,Sacramento Kings,Charlotte Hornets,Detroit Pistons,Boston Celtics
GAME_ID,0022300523,0022300518,0022300518,0022300519,0022300517
GAME_DATE,2024-01-10T00:00:00,2024-01-10T00:00:00,2024-01-10T00:00:00,2024-01-10T00:00:00,2024-01-10T00:00:00
MATCHUP,CHI vs. HOU,SAC @ CHA,CHA vs. SAC,DET vs. SAS,BOS vs. MIN
WL,W,W,L,L,W
MIN,53.0,48.0,48.0,48.0,53.0
FGM,42,46,35,44,39


## Get Data

In [3]:
ParallelPandas.initialize(n_cpu=8, split_factor=4, disable_pr_bar=True)

In [5]:
def get_final_shots_pbp(g):

    df = pd.DataFrame(game.PlayByPlay(game_id=g[0]).get_dict().get("game").get("actions")).assign(
        clock_minutes=lambda row: row["clock"].str.extract(
            "PT(\d{2})").astype(int),
        clock_seconds=lambda row: row["clock"].str.extract(
            "M(\d{2}.\d{2})S").astype(float),
        game_id=g[0],
        home_id=g[2],
        away_id=g[4],
        home_team=g[3],
        away_team=g[5],
        season=g[1]
    )

    dff = df.loc[
        (df["period"].isin([4, 5])) &
        (df["personId"] != 0),
        [
            "season", "game_id", "home_id", "away_id", "home_team", "away_team",
            "personId", "actionNumber", "period", "clock", "clock_minutes", "clock_seconds",
            "actionType", "subType", "shotResult", "scoreHome", "scoreAway",
            "x", "y", "shotDistance"
        ]
    ]
    
    player_team = nba_endpoints.boxscorescoringv2.BoxScoreScoringV2(g[0]).get_data_frames()[0][
        ["GAME_ID", "TEAM_ABBREVIATION", "PLAYER_ID", "PLAYER_NAME"]
    ]
    
    return dff.merge(
        player_team, left_on=["game_id", "personId"], right_on=["GAME_ID", "PLAYER_ID"]
    ).drop(["GAME_ID", "PLAYER_ID"], axis=1)
    
def get_final_shots_task(game):
    try:
        return get_final_shots_pbp(game)
    except Exception as e:
        print(e)
        return []

In [29]:
results = pd.DataFrame()
for g in tqdm(game_logs.values.tolist()[::-1]):
    shots = get_final_shots_task(g)
    if not isinstance(shots, list):
        if shots.shape[0] > 0:
            results = pd.concat([results, shots])
        else:
            pass
    else:
        pass

  0%|          | 0/6385 [00:00<?, ?it/s]

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)


In [35]:
results.columns = [
    "season", "game_id", "home_id", "away_id", "home_team", "away_team",
    "personId", "actionNumber", "period", "clock", "clock_minutes", "clock_seconds",
    "actionType", "subType", "shotResult", "scoreHome", "scoreAway",
    "x", "y", "shotDistance", "player_team", "player_name"
]

In [37]:
results

,season,game_id,home_id,away_id,home_team,away_team,personId,actionNumber,period,clock,clock_minutes,clock_seconds,actionType,subType,shotResult,scoreHome,scoreAway,x,y,shotDistance,player_team,player_name
0,2023-24,0022301230,1610612747,1610612740,LAL,NOP,1629629,504,4,PT12M00.00S,12,0.0,substitution,out,NaN,110,71,NaN,NaN,NaN,LAL,Cam Reddish
1,2023-24,0022301230,1610612747,1610612740,LAL,NOP,1629627,505,4,PT12M00.00S,12,0.0,substitution,out,NaN,110,71,NaN,NaN,NaN,NOP,Zion Williamson
2,2023-24,0022301230,1610612747,1610612740,LAL,NOP,1630230,506,4,PT12M00.00S,12,0.0,substitution,out,NaN,110,71,NaN,NaN,NaN,NOP,Naji Marshall
3,2023-24,0022301230,1610612747,1610612740,LAL,NOP,1627742,507,4,PT12M00.00S,12,0.0,substitution,out,NaN,110,71,NaN,NaN,NaN,NOP,Brandon Ingram
4,2023-24,0022301230,1610612747,1610612740,LAL,NOP,1631108,508,4,PT12M00.00S,12,0.0,substitution,in,NaN,110,71,NaN,NaN,NaN,LAL,Max Christie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,2019-20,0021900001,1610612761,1610612740,TOR,NOP,1628366,776,5,PT02M06.00S,2,6.0,foul,personal,NaN,122,122,NaN,NaN,NaN,NOP,Lonzo Ball
188,2019-20,0021900001,1610612761,1610612740,TOR,NOP,1628366,785,5,PT01M27.00S,1,27.0,substitution,out,NaN,127,122,NaN,NaN,NaN,NOP,Lonzo Ball
189,2019-20,0021900001,1610612761,1610612740,TOR,NOP,202324,766,5,PT03M08.00S,3,8.0,substitution,in,NaN,119,120,NaN,NaN,NaN,NOP,Derrick Favors
190,2019-20,0021900001,1610612761,1610612740,TOR,NOP,202324,772,5,PT02M37.00S,2,37.0,foul,personal,NaN,121,120,NaN,NaN,NaN,NOP,Derrick Favors


In [38]:
results[[
            "season", "game_id", "home_id", "away_id", "home_team", "away_team",
            "personId", "player_name", "player_team", "actionNumber", "period", 
            "clock", "clock_minutes", "clock_seconds", "actionType", "subType", 
            "shotResult", "scoreHome", "scoreAway", "x", "y", "shotDistance"
        ]].to_parquet(path="4Q_OT_Shots.parquet", index=False)

In [40]:
del results

In [4]:
shot_df = pd.read_parquet("4Q_OT_Shots.parquet")

In [5]:
shot_df.assign(
    shotResult_int=lambda row: np.where(row["shotResult"]=="Made", 1, 0)
).groupby("actionType").agg({
    "game_id": "count",
    "shotResult_int": "sum"
})

,game_id,shotResult_int
actionType,,
2pt,125337,65666
3pt,84222,28821
block,11530,0
ejection,142,0
foul,56382,0
freethrow,64604,49099
jumpball,1372,0
rebound,107347,0
steal,17001,0


In [6]:
shot_df["home_player"] = (shot_df["home_team"] == shot_df["player_team"]).astype(int)
shot_df["away_player"] = (shot_df["away_team"] == shot_df["player_team"]).astype(int)
shot_df["home_lead"] = (shot_df["scoreHome"] > shot_df["scoreAway"]).astype(int)
shot_df["away_lead"] = (shot_df["scoreHome"] < shot_df["scoreAway"]).astype(int)
shot_df["tied"] = (shot_df["scoreHome"] == shot_df["scoreAway"]).astype(int)
shot_df["shotResult_int"] = np.where(shot_df["shotResult"]=="Made", 1, 0)

In [7]:
shot_df.loc[shot_df["actionType"] != "substitution"]

,season,game_id,home_id,away_id,home_team,away_team,personId,player_name,player_team,actionNumber,...,scoreAway,x,y,shotDistance,home_player,away_player,home_lead,away_lead,tied,shotResult_int
5,2023-24,0022301230,1610612747,1610612740,LAL,NOP,1631108,Max Christie,LAL,545,...,79,33.722076,59.803922,26.90,1,0,0,1,0,0
6,2023-24,0022301230,1610612747,1610612740,LAL,NOP,1631108,Max Christie,LAL,560,...,81,NaN,NaN,NaN,1,0,0,1,0,0
7,2023-24,0022301230,1610612747,1610612740,LAL,NOP,1631108,Max Christie,LAL,565,...,81,NaN,NaN,NaN,1,0,0,1,0,0
8,2023-24,0022301230,1610612747,1610612740,LAL,NOP,1631108,Max Christie,LAL,567,...,81,NaN,NaN,NaN,1,0,0,1,0,0
9,2023-24,0022301230,1610612747,1610612740,LAL,NOP,1631108,Max Christie,LAL,578,...,81,NaN,NaN,NaN,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653763,2019-20,0021900001,1610612761,1610612740,TOR,NOP,200755,JJ Redick,NOP,804,...,122,NaN,NaN,NaN,0,1,1,0,0,0
653765,2019-20,0021900001,1610612761,1610612740,TOR,NOP,1628366,Lonzo Ball,NOP,759,...,120,91.672142,8.157169,21.15,0,1,0,1,0,0
653766,2019-20,0021900001,1610612761,1610612740,TOR,NOP,1628366,Lonzo Ball,NOP,762,...,120,NaN,NaN,NaN,0,1,0,1,0,0
653767,2019-20,0021900001,1610612761,1610612740,TOR,NOP,1628366,Lonzo Ball,NOP,776,...,122,NaN,NaN,NaN,0,1,0,0,1,0


In [39]:
len(shot_df.loc[
    (shot_df["actionType"].isin(["2pt", "3pt"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 30)
    & (np.abs(shot_df["scoreHome"].astype(int) - shot_df["scoreAway"].astype(int)) <= 3)
#     & (shot_df["player_name"] == "Derrick White")
, "game_id"
].unique())

1323

## Analysis

### Jump Shots

#### Shots with <= 30 seconds left in 4th or OT, one possession game

In [59]:
shot_df.loc[
    (shot_df["actionType"].isin(["2pt", "3pt"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 30)
    & (np.abs(shot_df["scoreHome"].astype(int) - shot_df["scoreAway"].astype(int)) <= 3)
#     & (shot_df["player_name"] == "Derrick White")
].groupby(["personId", "player_name", "actionType"]).agg({
    "shotResult_int": "sum",
    "game_id": "count",
    "shotDistance": "mean"
}).reset_index().pivot_table(
    index=["player_name"],
    columns=["actionType"],
    values=["shotDistance", "shotResult_int", "game_id"]
).assign(
    fgm=lambda row: (row[("shotResult_int", "2pt")] + row[("shotResult_int", "3pt")]),
    fga=lambda row: (row[("game_id", "2pt")] + row[("game_id", "3pt")]),
    avg_shotDistance=lambda row: ((row[("shotDistance", "2pt")]*row[("game_id", "2pt")])+(row[("shotDistance", "3pt")]*row[("game_id", "3pt")]))/row[("fga", "")],
    fg_perc=lambda row: row[("fgm", "")]/row[("fga", "")],
    efg_perc=lambda row: (row[("fgm", "")] + (0.5 * row[("shotResult_int", "3pt")]))/row[("fga", "")],
).sort_values(
    ("fga",""), ascending=False
).head(50).sort_values(("efg_perc",""), ascending=False).reset_index()

player_name game_id       shotDistance             \
actionType                              2pt   3pt          2pt        3pt   
0                        Kyle Kuzma    10.0  17.0     4.923000  28.006471   
1                       Paul George     5.0  19.0    10.296000  25.922105   
2                       Zach LaVine    26.0  15.0     6.687692  27.985333   
3                   Devonte' Graham     5.0  21.0     4.868000  29.865714   
4                       Buddy Hield    11.0  13.0     6.936364  28.418462   
5                    Damian Lillard    14.0  20.0     9.005714  31.015500   
6                  D'Angelo Russell     9.0  16.0     6.186667  26.694375   
7                      Kyrie Irving     5.0  16.0     7.244000  27.731875   
8                      Nikola Jokic    33.0  11.0     7.357879  29.639091   
9           Shai Gilgeous-Alexander    31.0  17.0     6.723226  27.593529   
10                  Dejounte Murray    17.0   9.0    10.151176  28.051111   
11                    Stephen Curry    15.0  23.0     7.580667  27.593043   
12                      Tyler Herro     8.0  23.0     9.597500  27.058261   
13                       Chris Paul    17.0   7.0     8.981765  30.737143   
14                     Jaylen Brown    17.0  11.0     8.982353  27.405455   
15            Giannis Antetokounmpo    21.0   2.0     4.868095  25.745000   
16                     De'Aaron Fox    36.0  10.0    10.026667  27.723000   
17                      Luka Doncic    20.0  29.0     5.773500  30.425172   
18                  Harrison Barnes    13.0  12.0     3.860000  26.135833   
19                     Jamal Murray    14.0  11.0    15.739286  29.415455   
20                Tyrese Haliburton     8.0  17.0     6.065000  29.928235   
21                    Collin Sexton    13.0   7.0     2.909231  28.501429   
22                     Jayson Tatum    36.0  14.0    12.551111  26.349286   
23                  Malcolm Brogdon    14.0   9.0     6.402143  30.458889   
24                        Ja Morant    30.0   2.0     4.083667  25.805000   
25                     Terry Rozier    10.0  20.0    14.762000  27.208000   
26                      Joel Embiid    30.0  11.0    11.611333  31.810000   
27                     Bradley Beal    25.0  12.0     8.850800  26.267500   
28                Spencer Dinwiddie    12.0  11.0     9.398333  27.042727   
29                    DeMar DeRozan    58.0  10.0    10.484138  33.375000   
30                 Donovan Mitchell    31.0  19.0     8.909355  32.019474   
31                      CJ McCollum    14.0  11.0     9.868571  26.192727   
32                   Brandon Ingram    15.0  10.0    14.568667  29.129000   
33                       RJ Barrett    14.0  10.0     5.824286  26.424000   
34                     Jerami Grant    11.0   9.0     5.745455  29.183333   
35                    Jalen Brunson    19.0   6.0     9.002105  27.226667   
36                    Pascal Siakam    19.0   5.0     6.498947  28.400000   
37                     Devin Booker    22.0  20.0    11.241364  26.350000   
38                       Trae Young    28.0  16.0     8.454643  32.495000   
39                Russell Westbrook    18.0  14.0     7.705000  25.640714   
40                     LeBron James    24.0  27.0     6.976250  27.382593   
41                    Fred VanVleet     9.0  18.0    10.397778  28.507222   
42                     Kevin Durant    16.0   7.0    11.401875  27.367143   
43                  Jordan Clarkson    10.0  12.0    12.264000  26.098333   
44                     Marcus Smart    14.0  10.0     5.842857  25.668000   
45                   Darius Garland    10.0  10.0     7.211000  30.688000   
46                    Julius Randle    23.0   7.0     8.094783  27.188571   
47                  Khris Middleton     5.0  17.0    11.362000  26.520588   
48                     James Harden    12.0  12.0     6.590000  27.275000   
49                     Jimmy Butler    28.0  18.0     9.799643  27.366111   

           shotResu

#### Shots with <= 60 seconds left in 4th or OT, one possession game

In [70]:
shot_df.loc[
    (shot_df["actionType"].isin(["2pt", "3pt"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 60)
    & (np.abs(shot_df["scoreHome"].astype(int) - shot_df["scoreAway"].astype(int)) <= 3)
#     & (shot_df["player_name"] == "Derrick White")
].groupby(["personId", "player_name", "actionType"]).agg({
    "shotResult_int": "sum",
    "game_id": "count",
    "shotDistance": "mean"
}).reset_index().pivot_table(
    index=["player_name"],
    columns=["actionType"],
    values=["shotDistance", "shotResult_int", "game_id"]
).assign(
    fgm=lambda row: (row[("shotResult_int", "2pt")] + row[("shotResult_int", "3pt")]),
    fga=lambda row: (row[("game_id", "2pt")] + row[("game_id", "3pt")]),
    avg_shotDistance=lambda row: ((row[("shotDistance", "2pt")]*row[("game_id", "2pt")])+(row[("shotDistance", "3pt")]*row[("game_id", "3pt")]))/row[("fga", "")],
    fg_perc=lambda row: row[("fgm", "")]/row[("fga", "")],
    efg_perc=lambda row: (row[("fgm", "")] + (0.5 * row[("shotResult_int", "3pt")]))/row[("fga", "")],
).sort_values(
    ("fga",""), 
    ascending=False
).head(50).sort_values(("efg_perc",""), ascending=False).reset_index()

player_name game_id       shotDistance             \
actionType                              2pt   3pt          2pt        3pt   
0                       Paul George    19.0  27.0     5.875263  25.745556   
1                      Jaylen Brown    27.0  15.0     8.395185  26.914667   
2                   Devonte' Graham     8.0  25.0     6.543750  29.545200   
3                        Kyle Kuzma    19.0  27.0     5.619474  27.711852   
4                   Dejounte Murray    33.0  11.0     8.738485  27.073636   
5                    Damian Lillard    33.0  32.0     5.875758  29.979063   
6                     Stephen Curry    31.0  35.0     6.037419  27.202857   
7             Giannis Antetokounmpo    31.0   3.0     5.143226  25.773333   
8                  D'Angelo Russell    13.0  22.0     8.319231  26.457273   
9           Shai Gilgeous-Alexander    40.0  21.0     6.379250  27.780952   
10                      Joel Embiid    44.0  16.0    11.160455  30.020625   
11                      Tyler Herro    16.0  32.0     9.524375  26.926875   
12                  Lauri Markkanen    21.0  12.0     5.720952  26.338333   
13                  Harrison Barnes    22.0  14.0     4.571818  26.182857   
14                      Luka Doncic    34.0  39.0     6.637941  30.558974   
15                     Nikola Jokic    52.0  15.0     7.256154  28.351333   
16                      Buddy Hield    15.0  25.0     7.062667  27.033200   
17                     Kyrie Irving    20.0  22.0     8.591000  27.382727   
18                      Zach LaVine    43.0  23.0     7.428605  27.314783   
19                     Terry Rozier    20.0  35.0    12.002000  26.617714   
20                    DeMar DeRozan    87.0  11.0     9.941149  32.499091   
21                     Bradley Beal    34.0  15.0     8.634412  26.222667   
22                     Jamal Murray    26.0  16.0    13.741923  29.601250   
23                     Jayson Tatum    56.0  25.0    10.542679  26.095600   
24                     De'Aaron Fox    61.0  13.0     9.551639  27.196154   
25                  Malcolm Brogdon    27.0  13.0     5.789630  29.470769   
26                Tyrese Haliburton    17.0  22.0     6.100588  29.559545   
27                       Kyle Lowry    27.0   9.0     6.552963  27.872222   
28                       Chris Paul    38.0  12.0    11.204737  28.908333   
29                       Trae Young    54.0  33.0     7.754259  30.642424   
30                 Donovan Mitchell    57.0  26.0     8.513509  30.692692   
31                    Fred VanVleet    15.0  27.0    10.322000  28.031852   
32                       RJ Barrett    21.0  13.0     5.627619  26.386154   
33                     Devin Booker    35.0  33.0    11.436571  26.376970   
34                      CJ McCollum    27.0  14.0    10.904444  25.682143   
35                    Julius Randle    39.0   7.0     7.017692  27.188571   
36                   Brandon Ingram    27.0  15.0    14.183333  27.847333   
37                Spencer Dinwiddie    22.0  17.0     8.179091  26.907647   
38                    Jalen Brunson    25.0   8.0     8.606800  26.376250   
39                    Pascal Siakam    29.0   8.0     6.665517  27.133750   
40                        Ja Morant    50.0   8.0     4.527600  26.587500   
41                  Jordan Clarkson    19.0  19.0     9.640526  26.162105   
42                  Khris Middleton    10.0  27.0    12.889000  26.322222   
43                     Kevin Durant    21.0  15.0    11.183810  26.350667   
44                Russell Westbrook    38.0  21.0     7.195526  26.017143   
45                   Darius Garland    20.0  16.0     5.893500  28.691875   
46                     LeBron James    38.0  41.0     5.985263  27.049024   
47                     Marcus Smart    20.0  16.0     5.997000  26.004375   
48                     Jimmy Butler    42.0  22.0     8.790238  26.960909   
49                     James Harden    22.0  20.0     5.813636  27.366000   

           shotResu

#### Shots with <= 30 seconds left in 4th or OT, one possession lead, leading team player has ball

In [61]:
shot_df.loc[
    (shot_df["actionType"].isin(["2pt", "3pt"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 30)
    & (((shot_df["home_lead"] == 1) & (shot_df["home_player"] == 1)) | ((shot_df["away_lead"] == 1) & (shot_df["away_player"] == 1)))
    & (np.abs(shot_df["scoreHome"].astype(int) - shot_df["scoreAway"].astype(int)) <= 3)
].groupby(["personId", "player_name", "actionType"]).agg({
    "shotResult_int": "sum",
    "game_id": "count",
    "shotDistance": "mean"
}).reset_index().pivot_table(
    index=["player_name"],
    columns=["actionType"],
    values=["shotDistance", "shotResult_int", "game_id"]
).assign(
    fgm=lambda row: (row[("shotResult_int", "2pt")] + row[("shotResult_int", "3pt")]),
    fga=lambda row: (row[("game_id", "2pt")] + row[("game_id", "3pt")]),
    avg_shotDistance=lambda row: ((row[("shotDistance", "2pt")]*row[("game_id", "2pt")])+(row[("shotDistance", "3pt")]*row[("game_id", "3pt")]))/row[("fga", "")],
    fg_perc=lambda row: row[("fgm", "")]/row[("fga", "")],
    efg_perc=lambda row: (row[("fgm", "")] + (0.5 * row[("shotResult_int", "3pt")]))/row[("fga", "")],
).sort_values(("fga",""), ascending=False).head(50).sort_values(("efg_perc",""), ascending=False)

game_id      shotDistance            shotResult_int  \
actionType                  2pt  3pt          2pt        3pt            2pt   
player_name                                                                   
D'Angelo Russell            3.0  3.0     4.956667  25.420000            2.0   
Karl-Anthony Towns          2.0  4.0     6.205000  26.432500            2.0   
Kyle Kuzma                  3.0  4.0     8.310000  25.230000            1.0   
Shai Gilgeous-Alexander     6.0  5.0     8.965000  27.702000            6.0   
Luka Doncic                 3.0  7.0     7.083333  28.834286            2.0   
Russell Westbrook           6.0  3.0     6.228333  23.020000            5.0   
Jrue Holiday                5.0  2.0    12.028000  25.620000            3.0   
Chris Paul                  4.0  1.0    11.100000  27.070000            4.0   
Malik Monk                  3.0  2.0    15.503333  26.975000            1.0   
Bradley Beal                6.0  1.0    11.160000  24.680000            4.0   
Devonte' Graham             2.0  6.0     5.340000  38.500000            0.0   
Kyrie Irving                1.0  5.0     0.310000  25.976000            0.0   
Lauri Markkanen             2.0  4.0     6.625000  27.142500            0.0   
Damian Lillard              6.0  6.0     6.058333  27.221667            3.0   
Aaron Gordon                5.0  1.0     2.964000  24.310000            3.0   
DeMar DeRozan              17.0  2.0    13.070000  25.425000           11.0   
Tyler Herro                 5.0  8.0    10.438000  26.180000            2.0   
Jamal Murray                5.0  2.0    16.810000  25.990000            2.0   
John Collins                4.0  1.0     1.800000  22.870000            2.0   
Paul George                 1.0  5.0    14.710000  26.806000            1.0   
De'Aaron Fox                6.0  3.0    10.940000  27.570000            3.0   
Zach LaVine                 7.0  2.0     5.572857  26.620000            3.0   
Spencer Dinwiddie           7.0  2.0    11.615714  26.800000            3.0   
Nikola Jokic               10.0  4.0     7.089000  29.215000            6.0   
Jayson Tatum               10.0  4.0    12.606000  27.470000            6.0   
Terry Rozier                4.0  4.0    18.180000  26.985000            2.0   
Stephen Curry               9.0  4.0     8.353333  27.292500            5.0   
Luguentz Dort               4.0  1.0     1.727500  26.970000            3.0   
Nikola Vucevic              4.0  1.0     7.397500  26.610000            3.0   
Devin Booker                6.0  5.0    18.183333  26.342000            2.0   
Jordan Poole                3.0  3.0     3.143333  25.306667            2.0   
Jordan Clarkson             3.0  4.0    10.690000  26.072500            1.0   
Joel Embiid                 9.0  3.0    10.595556  24.696667            5.0   
Jalen Brunson               6.0  2.0    10.040000  26.820000            4.0   
Kevin Durant                5.0  2.0    14.550000  26.675000            2.0   
Mikal Bridges               4.0  3.0    13.725000  23.743333            2.0   
Donovan Mitchell           13.0  3.0     9.060769  26.710000            6.0   
RJ Barrett                  6.0  6.0     6.138333  26.350000            1.0   
CJ McCollum                 6.0  2.0    12.138333  25.100000            2.0   
Brandon Ingram              6.0  2.0    17.131667  25.500000            2.0   
Julius Randle               9.0  2.0     6.251111  25.535000            3.0   
James Harden                4.0  6.0     9.145000  27.923333            1.0   
Trae Young                 12.0  2.0     9.403333  26.330000            4.0   
Jimmy Butler               11.0  2.0    13.649091  24.285000            5.0   
Dejounte Murray             6.0  1.0    11.610000  25.290000            1.0   
LeBron James                7.0  5.0     9.094286  27.370000            4.0   
Dennis Schroder             5.0  1.0     6.640000  25.590000            2.0   
Fred VanVleet               2.0  5.0     7.910000  33.592000            0.

#### Shots with <= 60 seconds left in 4th or OT, one possession lead, leading team player has ball

In [67]:
shot_df.loc[
    (shot_df["actionType"].isin(["2pt", "3pt"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 60)
    & (((shot_df["home_lead"] == 1) & (shot_df["home_player"] == 1)) | ((shot_df["away_lead"] == 1) & (shot_df["away_player"] == 1)))
    & (np.abs(shot_df["scoreHome"].astype(int) - shot_df["scoreAway"].astype(int)) <= 3)
].groupby(["personId", "player_name", "actionType"]).agg({
    "shotResult_int": "sum",
    "game_id": "count",
    "shotDistance": "mean"
}).reset_index().pivot_table(
    index=["player_name"],
    columns=["actionType"],
    values=["shotDistance", "shotResult_int", "game_id"]
).assign(
    fgm=lambda row: (row[("shotResult_int", "2pt")] + row[("shotResult_int", "3pt")]),
    fga=lambda row: (row[("game_id", "2pt")] + row[("game_id", "3pt")]),
    avg_shotDistance=lambda row: ((row[("shotDistance", "2pt")]*row[("game_id", "2pt")])+(row[("shotDistance", "3pt")]*row[("game_id", "3pt")]))/row[("fga", "")],
    fg_perc=lambda row: row[("fgm", "")]/row[("fga", "")],
    efg_perc=lambda row: (row[("shotResult_int", "2pt")] + (0.5 + row[("shotResult_int", "3pt")]))/row[("fga", "")],
).sort_values(
    ("fga",""), ascending=False
).head(50).sort_values(("efg_perc",""), ascending=False).reset_index()

player_name game_id       shotDistance             \
actionType                              2pt   3pt          2pt        3pt   
0                   Lauri Markkanen     7.0   6.0     7.630000  26.708333   
1           Shai Gilgeous-Alexander     9.0   7.0     7.735556  27.250000   
2                     DeMar DeRozan    25.0   3.0    12.674800  24.863333   
3                      Bradley Beal     9.0   2.0    10.570000  25.480000   
4                     Tobias Harris     9.0   5.0     7.865556  23.970000   
5                    Damian Lillard    16.0   7.0     4.310000  26.881429   
6                      Jamal Murray    10.0   3.0    14.234000  25.846667   
7                   Harrison Barnes     8.0   4.0     5.357500  25.287500   
8                       Paul George     7.0   7.0     5.285714  26.111429   
9                      Jrue Holiday    10.0   6.0     9.723000  26.850000   
10                  Malcolm Brogdon     8.0   3.0     5.280000  27.926667   
11                      Joel Embiid    18.0   5.0    10.305000  24.938000   
12                    Stephen Curry    14.0   8.0     7.352857  26.966250   
13                    Jalen Brunson     9.0   3.0     9.077778  25.433333   
14                       Chris Paul    16.0   4.0    13.318750  26.687500   
15                   Brandon Ingram    11.0   4.0    17.238182  25.572500   
16                   Nikola Vucevic     9.0   4.0     5.906667  25.937500   
17                Russell Westbrook    14.0   5.0     7.113571  24.692000   
18                    Mikal Bridges     9.0   4.0     9.852222  23.822500   
19                    Julius Randle    19.0   2.0     6.602105  25.535000   
20                     Nikola Jokic    19.0   4.0     8.292105  29.215000   
21                       Kyle Kuzma     5.0   8.0     8.126000  26.368750   
22                     Kevin Durant     7.0   6.0    13.655714  25.820000   
23                      Tyler Herro     9.0  14.0     8.863333  26.544286   
24                Tyrese Haliburton     6.0   5.0     8.281667  29.584000   
25                  Devonte' Graham     4.0   7.0     9.297500  37.115714   
26                        Ja Morant    26.0   4.0     4.285000  26.217500   
27                     Jayson Tatum    19.0   9.0    10.603158  26.563333   
28                  Dejounte Murray    13.0   3.0    11.802308  23.546667   
29                      Buddy Hield     5.0   7.0     7.236000  25.260000   
30                  Dennis Schroder     8.0   4.0     9.730000  24.705000   
31                      Luka Doncic    10.0  13.0     8.842000  30.715385   
32                     De'Aaron Fox    15.0   4.0    10.825333  27.057500   
33                     Jimmy Butler    17.0   2.0    12.052353  24.285000   
34                     Devin Booker    13.0  11.0    14.400769  26.426364   
35                 Donovan Mitchell    24.0   7.0     9.679583  27.535714   
36                     Kyrie Irving     6.0   9.0     9.711667  26.068889   
37                     Terry Rozier    11.0   9.0    14.691818  26.724444   
38                Spencer Dinwiddie    14.0   4.0     9.314286  26.470000   
39                      Zach LaVine    16.0   2.0     9.157500  26.620000   
40                       Trae Young    22.0   7.0     8.870000  27.335714   
41                      CJ McCollum    12.0   5.0    13.226667  24.326000   
42                    Fred VanVleet     3.0  11.0    10.470000  30.010909   
43                  Khris Middleton     5.0   9.0    10.714000  26.748889   
44                       Kyle Lowry     9.0   6.0     8.560000  27.571667   
45                    Pascal Siakam    11.0   1.0     9.229091  27.020000   
46                       RJ Barrett     9.0   8.0     7.112222  26.176250   
47                     James Harden     6.0  11.0     7.856667  28.196364   
48                     LeBron James    11.0  11.0     8.120909  26.740909   
49                     Marcus Smart     8.0   4.0     6.277500  26.495000   

           shotResu

#### Shots with <= 30 seconds left in 4th or OT, one possession lead, trailing team player has ball

In [62]:
shot_df.loc[
    (shot_df["actionType"].isin(["2pt", "3pt"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 30)
    & (((shot_df["home_lead"] == 1) & (shot_df["home_player"] == 0)) | ((shot_df["away_lead"] == 1) & (shot_df["away_player"] == 0)))
    & (np.abs(shot_df["scoreHome"].astype(int) - shot_df["scoreAway"].astype(int)) <= 3)
].groupby(["personId", "player_name", "actionType"]).agg({
    "shotResult_int": "sum",
    "game_id": "count",
    "shotDistance": "mean"
}).reset_index().pivot_table(
    index=["player_name"],
    columns=["actionType"],
    values=["shotDistance", "shotResult_int", "game_id"]
).assign(
    fgm=lambda row: (row[("shotResult_int", "2pt")] + row[("shotResult_int", "3pt")]),
    fga=lambda row: (row[("game_id", "2pt")] + row[("game_id", "3pt")]),
    avg_shotDistance=lambda row: ((row[("shotDistance", "2pt")]*row[("game_id", "2pt")])+(row[("shotDistance", "3pt")]*row[("game_id", "3pt")]))/row[("fga", "")],
    fg_perc=lambda row: row[("fgm", "")]/row[("fga", "")],
    efg_perc=lambda row: (row[("fgm", "")] + (0.5 * row[("shotResult_int", "3pt")]))/row[("fga", "")],
).sort_values(("fga",""), ascending=False).head(50).sort_values(("efg_perc",""), ascending=False)

game_id       shotDistance            shotResult_int  \
actionType                  2pt   3pt          2pt        3pt            2pt   
player_name                                                                    
Dejounte Murray             5.0   7.0     2.292000  28.354286            3.0   
Paul George                 2.0  10.0     2.730000  25.611000            2.0   
Stephen Curry               4.0  13.0     2.145000  27.643077            3.0   
Zach LaVine                16.0  10.0     7.105000  28.790000            8.0   
Devonte' Graham             1.0  11.0     3.060000  26.321818            0.0   
Chris Paul                  9.0   3.0     8.044444  26.986667            3.0   
Nikola Jokic               16.0   6.0     5.416250  30.266667           10.0   
Kyle Kuzma                  7.0  12.0     3.471429  29.299167            6.0   
Giannis Antetokounmpo      10.0   1.0     5.654000  26.490000            6.0   
Anfernee Simons             7.0   5.0     4.270000  28.976000            5.0   
Damian Lillard              4.0  11.0    12.767500  32.754545            2.0   
Terry Rozier                2.0  11.0     3.005000  28.356364            2.0   
Brandon Ingram              6.0   8.0    12.756667  30.036250            1.0   
Malcolm Brogdon             6.0   5.0     3.066667  31.700000            4.0   
Jaylen Brown               10.0   4.0     7.493000  26.050000            4.0   
Joel Embiid                11.0   7.0     7.306364  35.621429            3.0   
Harrison Barnes             7.0   9.0     4.014286  26.496667            2.0   
Shai Gilgeous-Alexander    24.0   9.0     6.068750  27.535556           13.0   
Tyrese Haliburton           6.0  12.0     3.968333  31.036667            4.0   
Pascal Siakam              11.0   5.0     4.598182  28.400000            6.0   
Scottie Barnes              8.0   3.0     4.950000  27.840000            5.0   
De'Aaron Fox               23.0   6.0    10.028261  28.201667           10.0   
Trae Young                 12.0  12.0     7.911667  31.692500            7.0   
LeBron James                8.0  15.0     7.725000  27.091333            5.0   
Jayson Tatum                8.0   9.0     8.432500  25.988889            4.0   
Klay Thompson               3.0   8.0    15.073333  27.111250            0.0   
Darius Garland              3.0   8.0    12.280000  31.858750            0.0   
Fred VanVleet               6.0  10.0     9.928333  26.539000            2.0   
Buddy Hield                 5.0  10.0     4.604000  29.215000            3.0   
Luka Doncic                14.0  18.0     4.053571  31.738333           11.0   
Ja Morant                  12.0   1.0     3.928333  26.190000            5.0   
Tyler Herro                 3.0  14.0     8.196667  27.536429            2.0   
Donovan Mitchell           15.0  13.0     8.776000  31.485385            3.0   
Devin Booker               11.0   8.0     7.380000  26.336250            4.0   
CJ McCollum                 8.0   7.0     8.166250  25.572857            4.0   
Spencer Dinwiddie           4.0   7.0     4.565000  26.864286            1.0   
Marcus Smart                6.0   8.0     4.790000  25.305000            2.0   
Domantas Sabonis            9.0   4.0     4.570000  28.155000            0.0   
Bradley Beal               14.0   8.0     7.456429  26.578750            3.0   
Jordan Poole                3.0   8.0     3.016667  29.592500            2.0   
DeMar DeRozan              25.0   6.0     7.465200  38.285000            8.0   
Jerami Grant                5.0   7.0     5.778000  25.627143            2.0   
Jalen Brunson              10.0   4.0     8.617000  27.430000            4.0   
Khris Middleton             2.0  12.0    11.665000  26.366667            1.0   
Julius Randle               9.0   2.0     8.204444  32.020000            3.0   
Jordan Clarkson             5.0   6.0    13.406000  24.913333            0.0   
Desmond Bane                3.0   8.0     1.830000  34.101250            1.0   
Kawhi Leonard           

#### Shots with <= 60 seconds left in 4th or OT, one possession lead, trailing team player has ball

In [68]:
shot_df.loc[
    (shot_df["actionType"].isin(["2pt", "3pt"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 60)
    & (((shot_df["home_lead"] == 1) & (shot_df["home_player"] == 0)) | ((shot_df["away_lead"] == 1) & (shot_df["away_player"] == 0)))
    & (np.abs(shot_df["scoreHome"].astype(int) - shot_df["scoreAway"].astype(int)) <= 3)
].groupby(["personId", "player_name", "actionType"]).agg({
    "shotResult_int": "sum",
    "game_id": "count",
    "shotDistance": "mean"
}).reset_index().pivot_table(
    index=["player_name"],
    columns=["actionType"],
    values=["shotDistance", "shotResult_int", "game_id"]
).assign(
    fgm=lambda row: (row[("shotResult_int", "2pt")] + row[("shotResult_int", "3pt")]),
    fga=lambda row: (row[("game_id", "2pt")] + row[("game_id", "3pt")]),
    avg_shotDistance=lambda row: ((row[("shotDistance", "2pt")]*row[("game_id", "2pt")])+(row[("shotDistance", "3pt")]*row[("game_id", "3pt")]))/row[("fga", "")],
    fg_perc=lambda row: row[("fgm", "")]/row[("fga", "")],
    efg_perc=lambda row: (row[("shotResult_int", "2pt")] + (0.5 + row[("shotResult_int", "3pt")]))/row[("fga", "")],
).sort_values(
    ("fga",""), ascending=False
).head(50).sort_values(("efg_perc",""), ascending=False).reset_index()

player_name game_id       shotDistance             \
actionType                              2pt   3pt          2pt        3pt   
0                   Dejounte Murray    13.0   7.0     2.944615  28.354286   
1                       Paul George     8.0  15.0     2.481250  25.569333   
2                   Anthony Edwards    14.0   2.0     4.871429  24.615000   
3                      Kyrie Irving    10.0   9.0     7.245000  28.651111   
4                     Stephen Curry    12.0  18.0     3.439167  27.075000   
5                      Jaylen Brown    16.0   7.0     7.184375  25.990000   
6                      Franz Wagner     6.0  10.0     2.556667  26.236000   
7                       Joel Embiid    15.0  10.0     7.616667  32.917000   
8                   Malcolm Brogdon    10.0   7.0     4.042000  30.037143   
9                        Trae Young    25.0  21.0     6.614000  30.744286   
10                      Luka Doncic    20.0  21.0     3.998000  31.247619   
11                     Nikola Jokic    24.0   9.0     5.264583  28.653333   
12                  Anfernee Simons    11.0   5.0     4.659091  28.976000   
13                      Zach LaVine    22.0  16.0     5.890909  27.460625   
14                    Pascal Siakam    16.0   7.0     5.140625  27.150000   
15          Shai Gilgeous-Alexander    28.0  11.0     5.848929  28.171818   
16                  Harrison Barnes     9.0  10.0     4.140000  26.541000   
17                     De'Aaron Fox    36.0   8.0     9.063056  27.501250   
18                      CJ McCollum    15.0   7.0     9.046667  25.572857   
19                    Fred VanVleet    11.0  11.0     9.566364  26.564545   
20                       Kyle Kuzma    11.0  18.0     5.136364  28.537222   
21                       Chris Paul    15.0   5.0     8.873333  26.604000   
22                     Jayson Tatum    17.0  15.0     6.616471  25.880667   
23                     Marcus Smart     9.0   9.0     5.866667  25.442222   
24                Tyrese Haliburton     7.0  14.0     3.558571  30.374286   
25                     Terry Rozier     3.0  19.0     2.540000  27.261579   
26                 Donovan Mitchell    27.0  16.0     7.358519  30.395625   
27                       Alec Burks     9.0   8.0     5.721111  26.681250   
28                 Domantas Sabonis    13.0   4.0     4.064615  28.155000   
29                   Darius Garland     8.0   9.0     7.611250  31.338889   
30                     LeBron James    14.0  22.0     6.013571  26.844545   
31                   Damian Lillard    13.0  18.0     6.636923  31.032778   
32                    DeMar DeRozan    37.0   6.0     7.353784  38.285000   
33                     Bradley Beal    17.0  10.0     7.717647  26.448000   
34                     Devin Booker    13.0  14.0     8.464615  26.305000   
35                    Jalen Brunson    12.0   4.0     8.095833  27.430000   
36                   Brandon Ingram    11.0  11.0    12.266364  28.674545   
37                      Buddy Hield     6.0  16.0     4.073333  27.917500   
38                  Khris Middleton     3.0  14.0    14.483333  26.130000   
39                  Lauri Markkanen    12.0   5.0     5.065833  25.846000   
40                Spencer Dinwiddie     7.0  10.0     5.190000  26.800000   
41                        Ja Morant    17.0   1.0     4.660000  26.190000   
42                      Tyler Herro     6.0  16.0     9.333333  27.299375   
43                     Kevin Durant     8.0   8.0     6.182500  26.875000   
44                     Jamal Murray     9.0   8.0    11.041111  31.990000   
45                  Cade Cunningham    10.0   7.0     8.157000  30.188571   
46                     Jerami Grant     9.0   9.0     8.757778  25.410000   
47                  Jordan Clarkson    11.0   9.0     9.730909  25.166667   
48                     Jimmy Butler    16.0  10.0     3.925625  25.050000   
49                Russell Westbrook    17.0  11.0     6.218235  26.350000   

           shotResu

#### Shots with <= 30 seconds left in 4th or OT, tie game

In [71]:
shot_df.loc[
    (shot_df["actionType"].isin(["2pt", "3pt"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 30)
    & (shot_df["tied"]==1)
].groupby(["personId", "player_name", "actionType"]).agg({
    "shotResult_int": "sum",
    "game_id": "count",
    "shotDistance": "mean"
}).reset_index().pivot_table(
    index=["player_name"],
    columns=["actionType"],
    values=["shotDistance", "shotResult_int", "game_id"]
).assign(
    fgm=lambda row: (row[("shotResult_int", "2pt")] + row[("shotResult_int", "3pt")]),
    fga=lambda row: (row[("game_id", "2pt")] + row[("game_id", "3pt")]),
    avg_shotDistance=lambda row: ((row[("shotDistance", "2pt")]*row[("game_id", "2pt")])+(row[("shotDistance", "3pt")]*row[("game_id", "3pt")]))/row[("fga", "")],
    fg_perc=lambda row: row[("fgm", "")]/row[("fga", "")],
    efg_perc=lambda row: (row[("fgm", "")] + (0.5 * row[("shotResult_int", "3pt")]))/row[("fga", "")],
).sort_values(
    ("fga",""
    ), ascending=False).head(50).sort_values(("efg_perc",""), ascending=False).reset_index()

player_name game_id      shotDistance             \
actionType                              2pt  3pt          2pt        3pt   
0                       Buddy Hield     3.0  2.0    11.100000  26.165000   
1                        Malik Monk     2.0  3.0     0.970000  24.563333   
2                        RJ Barrett     3.0  1.0     1.660000  25.720000   
3                 Bogdan Bogdanovic     1.0  3.0     1.970000  24.626667   
4                      De'Aaron Fox     7.0  1.0     9.238571  25.310000   
5                 Russell Westbrook     2.0  2.0    11.235000  26.270000   
6                Reggie Bullock Jr.     1.0  3.0     2.410000  22.963333   
7                      Jamal Murray     5.0  4.0    15.164000  27.782500   
8                        Alec Burks     2.0  3.0     9.350000  27.953333   
9                       Zach LaVine     3.0  3.0     7.063333  26.213333   
10                 Donovan Mitchell     3.0  3.0     8.920000  39.643333   
11                 Bojan Bogdanovic     3.0  2.0    12.296667  25.750000   
12                   Damian Lillard     4.0  3.0     9.665000  32.226667   
13            Giannis Antetokounmpo     4.0  1.0     2.545000  25.000000   
14                      Paul George     2.0  4.0    15.655000  25.595000   
15                     Nikola Jokic     7.0  1.0    12.180000  27.570000   
16                 D'Angelo Russell     4.0  8.0     9.632500  27.157500   
17                      Luka Doncic     3.0  4.0    12.490000  27.300000   
18                    Collin Sexton     4.0  3.0     1.265000  27.536667   
19                     Jayson Tatum    18.0  1.0    14.351111  25.110000   
20                     Bradley Beal     5.0  3.0     9.984000  25.966667   
21                  Devonte' Graham     2.0  4.0     5.300000  26.660000   
22                     Jaylen Brown     4.0  6.0    13.825000  28.993333   
23                    Anthony Davis     3.0  2.0     0.593333  25.215000   
24                  Khris Middleton     1.0  3.0    16.060000  26.250000   
25          Shai Gilgeous-Alexander     1.0  3.0     8.980000  27.586667   
26                    Fred VanVleet     1.0  3.0    18.190000  26.593333   
27                        Ja Morant     5.0  1.0     6.376000  25.420000   
28                     Jimmy Butler     7.0  8.0    11.062857  30.755000   
29                    DeMar DeRozan    16.0  2.0    12.453750  26.595000   
30                      Joel Embiid    10.0  1.0    17.261000  26.470000   
31                     Marcus Smart     3.0  2.0     5.640000  27.120000   
32                   Darius Garland     4.0  1.0     5.350000  26.660000   
33                  Dejounte Murray     6.0  1.0    15.241667  28.690000   
34                       Chris Paul     4.0  3.0     8.972500  35.710000   
35                     LeBron James     9.0  7.0     4.663333  28.015714   
36                     Terry Rozier     4.0  5.0    17.222500  24.860000   
37                  Malcolm Brogdon     6.0  2.0     9.023333  29.205000   
38                     Kyrie Irving     2.0  2.0     9.230000  27.985000   
39                 Kevin Porter Jr.     2.0  2.0     6.550000  26.445000   
40                     Devin Booker     5.0  7.0    11.406000  26.371429   
41                  Anthony Edwards     1.0  4.0     1.590000  28.122500   
42                     Caris LeVert     2.0  3.0     4.555000  25.580000   
43                    Julius Randle     5.0  3.0    11.216000  25.070000   
44                    Stephen Curry     2.0  6.0    14.975000  27.685000   
45                       Trae Young     4.0  2.0     7.237500  43.475000   
46                   Andrew Wiggins     3.0  3.0    13.070000  26.543333   
47                     Kevin Durant     5.0  1.0    15.056000  25.340000   
48                     James Harden     5.0  4.0     5.934000  27.017500   
49                  Jordan Clarkson     2.0  2.0    11.770000  29.705000   

           shotResult_int       fgm   fga avg_shotDistance   fg_perc  e

#### Shots with <= 60 seconds left in 4th or OT, tie game

In [64]:
shot_df.loc[
    (shot_df["actionType"].isin(["2pt", "3pt"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 60)
    & (shot_df["tied"]==1)
].groupby(["personId", "player_name", "actionType"]).agg({
    "shotResult_int": "sum",
    "game_id": "count",
    "shotDistance": "mean"
}).reset_index().pivot_table(
    index=["player_name"],
    columns=["actionType"],
    values=["shotDistance", "shotResult_int", "game_id"]
).assign(
    fgm=lambda row: (row[("shotResult_int", "2pt")] + row[("shotResult_int", "3pt")]),
    fga=lambda row: (row[("game_id", "2pt")] + row[("game_id", "3pt")]),
    avg_shotDistance=lambda row: ((row[("shotDistance", "2pt")]*row[("game_id", "2pt")])+(row[("shotDistance", "3pt")]*row[("game_id", "3pt")]))/row[("fga", "")],
    fg_perc=lambda row: row[("fgm", "")]/row[("fga", "")],
    efg_perc=lambda row: (row[("fgm", "")] + (0.5 * row[("shotResult_int", "3pt")]))/row[("fga", "")],
).sort_values(("fga",""), ascending=False).head(50).sort_values(("efg_perc",""), ascending=False)

game_id       shotDistance            shotResult_int  \
actionType                2pt   3pt          2pt        3pt            2pt   
player_name                                                                  
Malik Monk                3.0   3.0     0.993333  24.563333            3.0   
RJ Barrett                4.0   2.0     1.427500  26.595000            3.0   
Luka Doncic               4.0   5.0    14.327500  27.260000            3.0   
Miles Bridges             1.0   5.0     9.470000  35.114000            1.0   
Bogdan Bogdanovic         2.0   4.0     2.490000  24.735000            1.0   
Seth Curry                3.0   3.0    10.240000  26.273333            1.0   
Damian Lillard            4.0   7.0     9.665000  30.367143            2.0   
Jamal Murray              7.0   5.0    16.511429  28.032000            1.0   
D'Angelo Russell          4.0   9.0     9.632500  27.101111            3.0   
Devonte' Graham           2.0   5.0     5.300000  26.886000            1.0   
Scottie Barnes            5.0   2.0     2.698000  24.955000            1.0   
Giannis Antetokounmpo     7.0   1.0     4.511429  25.000000            3.0   
Nikola Jokic              9.0   2.0    10.380000  25.265000            6.0   
De'Aaron Fox             10.0   1.0     9.400000  25.310000            6.0   
Donovan Mitchell          6.0   3.0     9.046667  39.643333            3.0   
Paul George               4.0   5.0    13.695000  25.762000            0.0   
Russell Westbrook         7.0   5.0     9.732857  26.610000            3.0   
Zach LaVine               5.0   5.0     8.662000  27.126000            2.0   
Bradley Beal              8.0   3.0     8.405000  25.966667            4.0   
DeMar DeRozan            25.0   2.0    11.036800  26.595000           13.0   
Terry Rozier              6.0   7.0    11.801667  24.732857            3.0   
Jayson Tatum             20.0   1.0    13.822500  25.110000            8.0   
Collin Sexton             5.0   3.0     1.552000  27.536667            2.0   
Tyrese Haliburton         4.0   3.0     7.277500  25.716667            3.0   
Khris Middleton           2.0   4.0    15.935000  26.035000            1.0   
Darius Garland            8.0   4.0     4.820000  25.720000            2.0   
Chris Paul                7.0   3.0    11.368571  35.710000            4.0   
Jaylen Brown              7.0   6.0    10.921429  28.993333            2.0   
Bojan Bogdanovic          4.0   3.0    10.430000  26.233333            1.0   
Stephen Curry             5.0   9.0     8.590000  27.668889            2.0   
Jordan Clarkson           3.0   4.0     9.370000  28.267500            1.0   
LeBron James             13.0   8.0     4.147692  28.035000            6.0   
Jimmy Butler              9.0  10.0    11.276667  29.407000            2.0   
Anthony Davis             3.0   3.0     0.593333  25.316667            2.0   
Kawhi Leonard             4.0   2.0     6.772500  39.035000            2.0   
Ja Morant                 7.0   3.0     5.107143  27.213333            3.0   
Joel Embiid              11.0   1.0    17.392727  26.470000            2.0   
Malcolm Brogdon           9.0   3.0     8.184444  29.693333            2.0   
Kevin Porter Jr.          3.0   4.0     5.323333  26.210000            2.0   
Kyle Lowry                5.0   2.0     6.756000  28.430000            2.0   
Devin Booker              9.0   8.0    11.447778  26.435000            3.0   
Kyrie Irving              4.0   4.0    10.275000  27.485000            2.0   
Dejounte Murray           7.0   1.0    13.808571  28.690000            2.0   
James Harden              7.0   5.0     6.262857  26.772000            1.0   
Trae Young                7.0   5.0     8.320000  34.844000            1.0   
Andrew Wiggins            3.0   3.0    13.070000  26.543333            1.0   
Julius Randle             7.0   3.0    10.142857  25.070000            0.0   
Kevin Durant              6.0   1.0    14.968333  25.340000            1.0   
Caris LeVert              4.0   4.0     6.61

### Free Throws

#### FTs with <= 30 seconds left in 4th or OT, one possession game

In [54]:
shot_df.loc[
    (shot_df["actionType"].isin(["freethrow"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 30)
    & (np.abs(shot_df["scoreHome"].astype(int) - shot_df["scoreAway"].astype(int)) <= 3)
].groupby(["personId", "player_name"]).agg({
    "shotResult_int": "sum",
    "game_id": "count"
}).reset_index().assign(
    ft_perc=lambda row: row["shotResult_int"]/row["game_id"]
).sort_values("game_id", ascending=False).head(50).sort_values("ft_perc", ascending=False)

,personId,player_name,shotResult_int,game_id,ft_perc
175,1628374,Lauri Markkanen,16,17,0.941176
103,203924,Jerami Grant,20,22,0.909091
43,202681,Kyrie Irving,20,23,0.869565
60,203078,Bradley Beal,26,30,0.866667
2,101108,Chris Paul,19,22,0.863636
28,201939,Stephen Curry,19,22,0.863636
8,201142,Kevin Durant,27,32,0.843750
38,202331,Paul George,21,25,0.840000
112,203954,Joel Embiid,25,30,0.833333
98,203897,Zach LaVine,29,35,0.828571


#### FTs with <= 60 seconds left in 4th or OT, one possession game

In [17]:
shot_df.loc[
    (shot_df["actionType"].isin(["freethrow"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 60)
    & (np.abs(shot_df["scoreHome"].astype(int) - shot_df["scoreAway"].astype(int)) <= 3)
].groupby(["personId", "player_name"]).agg({
    "shotResult_int": "sum",
    "game_id": "count"
}).reset_index().assign(
    fg_perc=lambda row: row["shotResult_int"]/row["game_id"]
).sort_values("game_id", ascending=False).head(50).sort_values("fg_perc", ascending=False)

,personId,player_name,shotResult_int,game_id,fg_perc
47,202681,Kyrie Irving,26,29,0.896552
31,201939,Stephen Curry,25,28,0.892857
111,203924,Jerami Grant,23,26,0.884615
2,101108,Chris Paul,22,25,0.880000
106,203897,Zach LaVine,38,44,0.863636
9,201142,Kevin Durant,30,35,0.857143
42,202331,Paul George,24,28,0.857143
67,203078,Bradley Beal,36,42,0.857143
221,1628983,Shai Gilgeous-Alexander,46,54,0.851852
68,203081,Damian Lillard,43,51,0.843137


#### FTs with <= 30 seconds left in 4th or OT, one possession lead, leading team player has ball

In [18]:
shot_df.loc[
    (shot_df["actionType"].isin(["freethrow"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 30)
    & (((shot_df["home_lead"] == 1) & (shot_df["home_player"] == 1)) | ((shot_df["away_lead"] == 1) & (shot_df["away_player"] == 1)))
    & (np.abs(shot_df["scoreHome"].astype(int) - shot_df["scoreAway"].astype(int)) <= 3)
].groupby(["personId", "player_name"]).agg({
    "shotResult_int": "sum",
    "game_id": "count",
}).reset_index().assign(
    fg_perc=lambda row: row["shotResult_int"]/row["game_id"]
).sort_values("game_id", ascending=False).head(50).sort_values("fg_perc", ascending=False)

,personId,player_name,shotResult_int,game_id,fg_perc
248,1630175,Cole Anthony,10,10,1.000000
2,101108,Chris Paul,16,17,0.941176
7,201142,Kevin Durant,21,23,0.913043
56,203078,Bradley Beal,19,21,0.904762
57,203081,Damian Lillard,23,26,0.884615
151,1628368,De'Aaron Fox,15,17,0.882353
90,203897,Zach LaVine,12,14,0.857143
94,203924,Jerami Grant,11,13,0.846154
192,1629001,De'Anthony Melton,11,13,0.846154
25,201939,Stephen Curry,16,19,0.842105


#### FTs with <= 60 seconds left in 4th or OT, one possession lead, leading team player has ball

In [19]:
shot_df.loc[
    (shot_df["actionType"].isin(["freethrow"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 60)
    & (((shot_df["home_lead"] == 1) & (shot_df["home_player"] == 1)) | ((shot_df["away_lead"] == 1) & (shot_df["away_player"] == 1)))
    & (np.abs(shot_df["scoreHome"].astype(int) - shot_df["scoreAway"].astype(int)) <= 3)
].groupby(["personId", "player_name"]).agg({
    "shotResult_int": "sum",
    "game_id": "count",
}).reset_index().assign(
    fg_perc=lambda row: row["shotResult_int"]/row["game_id"]
).sort_values("game_id", ascending=False).head(50).sort_values("fg_perc", ascending=False)

,personId,player_name,shotResult_int,game_id,fg_perc
2,101108,Chris Paul,18,19,0.947368
60,203078,Bradley Beal,26,28,0.928571
8,201142,Kevin Durant,23,25,0.920000
96,203897,Zach LaVine,18,20,0.900000
61,203081,Damian Lillard,23,26,0.884615
161,1628368,De'Aaron Fox,15,17,0.882353
100,203924,Jerami Grant,12,14,0.857143
27,201939,Stephen Curry,18,21,0.857143
215,1629027,Trae Young,29,34,0.852941
202,1629001,De'Anthony Melton,11,13,0.846154


#### FTs with <= 30 seconds left in 4th or OT, one possession lead, trailing team player has ball

In [20]:
shot_df.loc[
    (shot_df["actionType"].isin(["freethrow"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 30)
    & (((shot_df["home_lead"] == 1) & (shot_df["home_player"] == 0)) | ((shot_df["away_lead"] == 1) & (shot_df["away_player"] == 0)))
    & (np.abs(shot_df["scoreHome"].astype(int) - shot_df["scoreAway"].astype(int)) <= 3)
].groupby(["personId", "player_name"]).agg({
    "shotResult_int": "sum",
    "game_id": "count"
}).reset_index().assign(
    fg_perc=lambda row: row["shotResult_int"]/row["game_id"]
).sort_values("game_id", ascending=False).head(50).sort_values("fg_perc", ascending=False)

,personId,player_name,shotResult_int,game_id,fg_perc
20,202681,Kyrie Irving,5,5,1.000000
59,203924,Jerami Grant,7,7,1.000000
128,1629014,Anfernee Simons,5,5,1.000000
107,1628374,Lauri Markkanen,10,10,1.000000
158,1630169,Tyrese Haliburton,7,7,1.000000
124,1628983,Shai Gilgeous-Alexander,15,16,0.937500
19,202331,Paul George,8,9,0.888889
24,202692,Alec Burks,7,8,0.875000
86,1627742,Brandon Ingram,6,7,0.857143
145,1629636,Darius Garland,6,7,0.857143


#### FTs with <= 60 seconds left in 4th or OT, one possession lead, trailing team player has ball

In [21]:
shot_df.loc[
    (shot_df["actionType"].isin(["freethrow"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 60)
    & (((shot_df["home_lead"] == 1) & (shot_df["home_player"] == 0)) | ((shot_df["away_lead"] == 1) & (shot_df["away_player"] == 0)))
    & (np.abs(shot_df["scoreHome"].astype(int) - shot_df["scoreAway"].astype(int)) <= 3)
].groupby(["personId", "player_name"]).agg({
    "shotResult_int": "sum",
    "game_id": "count"
}).reset_index().assign(
    fg_perc=lambda row: row["shotResult_int"]/row["game_id"]
).sort_values("game_id", ascending=False).head(50).sort_values("fg_perc", ascending=False)

,personId,player_name,shotResult_int,game_id,fg_perc
28,202681,Kyrie Irving,10,10,1.000000
126,1628374,Lauri Markkanen,12,12,1.000000
190,1630169,Tyrese Haliburton,8,8,1.000000
148,1628983,Shai Gilgeous-Alexander,20,21,0.952381
169,1629628,RJ Barrett,11,12,0.916667
72,203924,Jerami Grant,9,10,0.900000
68,203903,Jordan Clarkson,9,10,0.900000
33,202692,Alec Burks,9,10,0.900000
101,1627742,Brandon Ingram,9,10,0.900000
173,1629636,Darius Garland,9,10,0.900000


#### FTs with <= 30 seconds left in 4th or OT, tie game

In [22]:
shot_df.loc[
    (shot_df["actionType"].isin(["freethrow"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 30)
    & (shot_df["tied"]==1)
].groupby(["personId", "player_name"]).agg({
    "shotResult_int": "sum",
    "game_id": "count"
}).reset_index().assign(
    fg_perc=lambda row: row["shotResult_int"]/row["game_id"]
).sort_values("game_id", ascending=False).head(50).sort_values("fg_perc", ascending=False)

,personId,player_name,shotResult_int,game_id,fg_perc
41,1626164,Devin Booker,2,2,1.000000
44,1627734,Domantas Sabonis,3,3,1.000000
12,201937,Ricky Rubio,2,2,1.000000
85,1629636,Darius Garland,2,2,1.000000
77,1629014,Anfernee Simons,2,2,1.000000
51,1627759,Jaylen Brown,2,2,1.000000
24,203081,Damian Lillard,2,2,1.000000
25,203114,Khris Middleton,2,2,1.000000
96,1631095,Jabari Smith Jr.,1,1,1.000000
4,201566,Russell Westbrook,3,3,1.000000


#### FTs with <= 60 seconds left in 4th or OT, tie game

In [23]:
shot_df.loc[
    (shot_df["actionType"].isin(["freethrow"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 60)
    & (shot_df["tied"]==1)
].groupby(["personId", "player_name"]).agg({
    "shotResult_int": "sum",
    "game_id": "count"
}).reset_index().assign(
    fg_perc=lambda row: row["shotResult_int"]/row["game_id"]
).sort_values("game_id", ascending=False).head(50).sort_values("fg_perc", ascending=False)

,personId,player_name,shotResult_int,game_id,fg_perc
126,1630567,Scottie Barnes,3,3,1.000000
30,203081,Damian Lillard,3,3,1.000000
55,1626179,Terry Rozier,3,3,1.000000
127,1630578,Alperen Sengun,2,2,1.000000
67,1627759,Jaylen Brown,2,2,1.000000
128,1630595,Cade Cunningham,2,2,1.000000
16,202331,Paul George,4,4,1.000000
78,1628374,Lauri Markkanen,4,4,1.000000
5,201566,Russell Westbrook,3,3,1.000000
129,1631094,Paolo Banchero,3,3,1.000000


### Last Shot

#### Last shot of the game %

In [65]:
shot_df.loc[
    (shot_df["actionType"].isin(["2pt", "3pt"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 30)
    & (shot_df["tied"]==1)
].groupby(["personId", "player_name", "actionType"]).agg({
    "shotResult_int": "sum",
    "game_id": "count",
    "shotDistance": "mean"
}).reset_index().pivot_table(
    index=["player_name"],
    columns=["actionType"],
    values=["shotDistance", "shotResult_int", "game_id"]
).assign(
    fgm=lambda row: (row[("shotResult_int", "2pt")] + row[("shotResult_int", "3pt")]),
    fga=lambda row: (row[("game_id", "2pt")] + row[("game_id", "3pt")]),
    avg_shotDistance=lambda row: ((row[("shotDistance", "2pt")]*row[("game_id", "2pt")])+(row[("shotDistance", "3pt")]*row[("game_id", "3pt")]))/row[("fga", "")],
    fg_perc=lambda row: row[("fgm", "")]/row[("fga", "")],
    efg_perc=lambda row: (row[("fgm", "")] + (0.5 * row[("shotResult_int", "3pt")]))/row[("fga", "")],
).sort_values(("fga",""), ascending=False).head(50)#.sort_values(("efg_perc",""), ascending=False)

game_id      shotDistance            shotResult_int  \
actionType                  2pt  3pt          2pt        3pt            2pt   
player_name                                                                   
Jayson Tatum               18.0  1.0    14.351111  25.110000            7.0   
DeMar DeRozan              16.0  2.0    12.453750  26.595000            6.0   
LeBron James                9.0  7.0     4.663333  28.015714            3.0   
Jimmy Butler                7.0  8.0    11.062857  30.755000            2.0   
D'Angelo Russell            4.0  8.0     9.632500  27.157500            3.0   
Devin Booker                5.0  7.0    11.406000  26.371429            1.0   
Joel Embiid                10.0  1.0    17.261000  26.470000            2.0   
Jaylen Brown                4.0  6.0    13.825000  28.993333            1.0   
James Harden                5.0  4.0     5.934000  27.017500            0.0   
Jamal Murray                5.0  4.0    15.164000  27.782500            1.0   
Terry Rozier                4.0  5.0    17.222500  24.860000            1.0   
Malcolm Brogdon             6.0  2.0     9.023333  29.205000            2.0   
Julius Randle               5.0  3.0    11.216000  25.070000            0.0   
Stephen Curry               2.0  6.0    14.975000  27.685000            0.0   
Bradley Beal                5.0  3.0     9.984000  25.966667            2.0   
Nikola Jokic                7.0  1.0    12.180000  27.570000            4.0   
De'Aaron Fox                7.0  1.0     9.238571  25.310000            5.0   
Luka Doncic                 3.0  4.0    12.490000  27.300000            2.0   
Dejounte Murray             6.0  1.0    15.241667  28.690000            2.0   
Damian Lillard              4.0  3.0     9.665000  32.226667            2.0   
Collin Sexton               4.0  3.0     1.265000  27.536667            2.0   
Chris Paul                  4.0  3.0     8.972500  35.710000            2.0   
Ja Morant                   5.0  1.0     6.376000  25.420000            2.0   
Devonte' Graham             2.0  4.0     5.300000  26.660000            1.0   
Andrew Wiggins              3.0  3.0    13.070000  26.543333            1.0   
Donovan Mitchell            3.0  3.0     8.920000  39.643333            2.0   
Kevin Durant                5.0  1.0    15.056000  25.340000            1.0   
Paul George                 2.0  4.0    15.655000  25.595000            0.0   
Trae Young                  4.0  2.0     7.237500  43.475000            1.0   
Zach LaVine                 3.0  3.0     7.063333  26.213333            2.0   
Marcus Smart                3.0  2.0     5.640000  27.120000            0.0   
Malik Monk                  2.0  3.0     0.970000  24.563333            2.0   
Alec Burks                  2.0  3.0     9.350000  27.953333            0.0   
Bojan Bogdanovic            3.0  2.0    12.296667  25.750000            1.0   
Buddy Hield                 3.0  2.0    11.100000  26.165000            2.0   
Giannis Antetokounmpo       4.0  1.0     2.545000  25.000000            1.0   
Caris LeVert                2.0  3.0     4.555000  25.580000            1.0   
Anthony Davis               3.0  2.0     0.593333  25.215000            2.0   
Anthony Edwards             1.0  4.0     1.590000  28.122500            1.0   
Darius Garland              4.0  1.0     5.350000  26.660000            0.0   
Kevin Porter Jr.            2.0  2.0     6.550000  26.445000            1.0   
Reggie Bullock Jr.          1.0  3.0     2.410000  22.963333            1.0   
Russell Westbrook           2.0  2.0    11.235000  26.270000            1.0   
Shai Gilgeous-Alexander     1.0  3.0     8.980000  27.586667            0.0   
Bogdan Bogdanovic           1.0  3.0     1.970000  24.626667            1.0   
Kyrie Irving                2.0  2.0     9.230000  27.985000            1.0   
RJ Barrett                  3.0  1.0     1.660000  25.720000            2.0   
Khris Middleton             1.0  3.0    16.060000  26.250000            0.

In [25]:
last_shot_base = shot_df.loc[
    (shot_df["actionType"].isin(["2pt", "3pt"]))
    & (shot_df["clock_minutes"] == 0)
    & (shot_df["clock_seconds"] <= 30)
].sort_values(["game_id", "period", "actionNumber"])

In [26]:
last_shot_base["shot_rev_order"] = last_shot_base.groupby(["game_id", "period"])["actionNumber"].rank(ascending=False)

In [27]:
last_shot_base["scoreHome_prev"] = last_shot_base.groupby(["game_id", "period"])["scoreHome"].shift(1)
last_shot_base["scoreAway_prev"] = last_shot_base.groupby(["game_id", "period"])["scoreAway"].shift(1)

In [28]:
last_shot_base

,season,game_id,home_id,away_id,home_team,away_team,personId,player_name,player_team,actionNumber,...,shotDistance,home_player,away_player,home_lead,away_lead,tied,shotResult_int,shot_rev_order,scoreHome_prev,scoreAway_prev
653638,2019-20,0021900001,1610612761,1610612740,TOR,NOP,201950,Jrue Holiday,NOP,727,...,24.88,0,1,0,0,1,0,2.0,NaN,NaN
653721,2019-20,0021900001,1610612761,1610612740,TOR,NOP,1626181,Norman Powell,TOR,734,...,28.37,1,0,0,0,1,0,1.0,117,117
653733,2019-20,0021900001,1610612761,1610612740,TOR,NOP,1627832,Fred VanVleet,TOR,798,...,1.96,1,0,1,0,0,0,2.0,NaN,NaN
653641,2019-20,0021900001,1610612761,1610612740,TOR,NOP,201950,Jrue Holiday,NOP,802,...,27.72,0,1,1,0,0,0,1.0,130,122
653545,2019-20,0021900002,1610612746,1610612747,LAC,LAL,203584,Troy Daniels,LAL,677,...,23.87,0,1,1,0,0,0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,2023-24,0022301227,1610612738,1610612752,BOS,NYK,203944,Julius Randle,NYK,646,...,1.82,0,1,1,0,0,1,1.0,NaN,NaN
374,2023-24,0022301228,1610612756,1610612758,PHX,SAC,1631165,Keon Ellis,SAC,663,...,26.76,0,1,0,1,0,0,1.0,NaN,NaN
157,2023-24,0022301229,1610612749,1610612754,MIL,IND,203114,Khris Middleton,MIL,693,...,23.63,1,0,0,1,0,1,2.0,NaN,NaN
158,2023-24,0022301229,1610612749,1610612754,MIL,IND,203114,Khris Middleton,MIL,699,...,2.87,1,0,0,1,0,0,1.0,119,126


In [109]:
last_shot_base.loc[
    (shot_df["actionType"].isin(["2pt", "3pt"]))
    & (last_shot_base["shot_rev_order"] <= 2)
    & (np.abs(last_shot_base["scoreHome"].astype(int) - last_shot_base["scoreAway"].astype(int)) <= 3)
#     & (last_shot_base["tied"]==1)
].assign(
    tied_or_took_lead=lambda row: (((row["scoreHome_prev"] <= row["scoreAway_prev"]) & (row["scoreHome"] >= row["scoreAway"]) & (row["home_player"] == 1)) | (
        (row["scoreAway_prev"] <= row["scoreHome_prev"]) & (row["scoreAway"] >= row["scoreHome"]) & (row["away_player"]==1))),
    tied_or_took_lead_made=lambda row: (((row["scoreHome_prev"] <= row["scoreAway_prev"]) & (row["scoreHome"] >= row["scoreAway"])) | (
        (row["scoreAway_prev"] <= row["scoreHome_prev"]) & (row["scoreAway"] >= row["scoreHome"]))) & (row["shotResult_int"]==1)
).groupby(["personId", "player_name"]).agg({
    "shotResult_int": "sum",
    "game_id": "count",
    "shotDistance": "mean",
    "tied_or_took_lead_made": "sum",
    "tied_or_took_lead": "sum",
}).reset_index().assign(
    fg_perc=lambda row: row["shotResult_int"]/row["game_id"],
    fg_perc_tie_or_take_lead=lambda row: row["tied_or_took_lead_made"]/row["tied_or_took_lead"],
).sort_values(
    "game_id", ascending=False
).head(23).sort_values("fg_perc", ascending=False).reset_index()


,index,personId,player_name,shotResult_int,game_id,shotDistance,tied_or_took_lead_made,tied_or_took_lead,fg_perc,fg_perc_tie_or_take_lead
0,186,1628368,De'Aaron Fox,13,25,16.800800,4,6,0.520000,0.666667
1,228,1628983,Shai Gilgeous-Alexander,18,35,14.958286,8,9,0.514286,0.888889
2,167,1627759,Jaylen Brown,11,23,16.980000,3,8,0.478261,0.375000
3,106,203897,Zach LaVine,13,28,15.455000,4,5,0.464286,0.800000
4,124,203999,Nikola Jokic,15,33,15.236364,6,8,0.454545,0.750000
5,157,1627741,Buddy Hield,9,20,17.000500,2,3,0.450000,0.666667
6,251,1629029,Luka Doncic,16,38,21.266842,5,9,0.421053,0.555556
7,70,203081,Damian Lillard,9,22,24.503636,4,6,0.409091,0.666667
8,187,1628369,Jayson Tatum,15,38,17.451316,6,14,0.394737,0.428571
9,32,201942,DeMar DeRozan,18,47,14.678723,6,14,0.382979,0.428571


### Redefining "Clutch" Games

In [30]:
clutch_games_base = shot_df.loc[
    (shot_df["period"] == 4)
    & (np.abs(shot_df["scoreHome"].astype(int) - shot_df["scoreAway"].astype(int)) <= 5)
    & (shot_df["clock_minutes"] < 5)
].sort_values(["game_id", "actionNumber"])
clutch_games_base["action_rank_asc"] = clutch_games_base.groupby(["game_id", "period"])["actionNumber"].rank(ascending=True)
# clutch_games_base["action_rank_desc"] = clutch_games_base.groupby(["game_id", "period"])["actionNumber"].rank(ascending=False)
clutch_games_base

,season,game_id,home_id,away_id,home_team,away_team,personId,player_name,player_team,actionNumber,...,x,y,shotDistance,home_player,away_player,home_lead,away_lead,tied,shotResult_int,action_rank_asc
653600,2019-20,0021900001,1610612761,1610612740,TOR,NOP,1627783,Pascal Siakam,TOR,649,...,8.492116,47.372855,3.53,1,0,0,1,0,0,1.0
653710,2019-20,0021900001,1610612761,1610612740,TOR,NOP,1626143,Jahlil Okafor,NOP,650,...,NaN,NaN,NaN,0,1,0,1,0,0,2.0
653601,2019-20,0021900001,1610612761,1610612740,TOR,NOP,1627783,Pascal Siakam,TOR,651,...,NaN,NaN,NaN,1,0,0,1,0,0,3.0
653602,2019-20,0021900001,1610612761,1610612740,TOR,NOP,1627783,Pascal Siakam,TOR,652,...,5.000000,50.000000,0.00,1,0,0,1,0,0,4.0
653603,2019-20,0021900001,1610612761,1610612740,TOR,NOP,1627783,Pascal Siakam,TOR,653,...,NaN,NaN,NaN,1,0,0,1,0,0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,2023-24,0022301229,1610612749,1610612754,MIL,IND,201572,Brook Lopez,MIL,672,...,66.310775,66.176471,27.63,1,0,0,1,0,0,41.0
273,2023-24,0022301229,1610612749,1610612754,MIL,IND,1626167,Myles Turner,IND,673,...,NaN,NaN,NaN,0,1,0,1,0,0,42.0
215,2023-24,0022301229,1610612749,1610612754,MIL,IND,201572,Brook Lopez,MIL,674,...,NaN,NaN,NaN,1,0,0,1,0,0,43.0
140,2023-24,0022301229,1610612749,1610612754,MIL,IND,1630169,Tyrese Haliburton,IND,676,...,37.270039,47.303922,29.81,0,1,0,1,0,0,44.0


In [31]:
clutch_games_base_end = shot_df.loc[
    (shot_df["period"] == 4)
].sort_values(["game_id", "actionNumber"])
clutch_games_base_end["action_rank_desc"] = clutch_games_base_end.groupby(["game_id", "period"])["actionNumber"].rank(ascending=False)
clutch_games_base_end

,season,game_id,home_id,away_id,home_team,away_team,personId,player_name,player_team,actionNumber,...,x,y,shotDistance,home_player,away_player,home_lead,away_lead,tied,shotResult_int,action_rank_desc
653580,2019-20,0021900001,1610612761,1610612740,TOR,NOP,1628384,OG Anunoby,TOR,546,...,NaN,NaN,NaN,1,0,1,0,0,0,139.0
653586,2019-20,0021900001,1610612761,1610612740,TOR,NOP,1629638,Nickeil Alexander-Walker,NOP,547,...,NaN,NaN,NaN,0,1,1,0,0,0,138.0
653587,2019-20,0021900001,1610612761,1610612740,TOR,NOP,202734,E'Twaun Moore,NOP,548,...,NaN,NaN,NaN,0,1,1,0,0,0,137.0
653588,2019-20,0021900001,1610612761,1610612740,TOR,NOP,1627783,Pascal Siakam,TOR,549,...,NaN,NaN,NaN,1,0,1,0,0,0,136.0
653614,2019-20,0021900001,1610612761,1610612740,TOR,NOP,1628402,Frank Jackson,NOP,550,...,NaN,NaN,NaN,0,1,1,0,0,0,135.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,2023-24,0022301230,1610612747,1610612740,LAL,NOP,1631108,Max Christie,LAL,648,...,17.821945,52.941176,11.59,1,0,0,1,0,1,5.0
38,2023-24,0022301230,1610612747,1610612740,LAL,NOP,1641722,Jordan Hawkins,NOP,649,...,75.377792,43.627451,18.17,0,1,0,1,0,0,4.0
111,2023-24,0022301230,1610612747,1610612740,LAL,NOP,1641721,Maxwell Lewis,LAL,650,...,NaN,NaN,NaN,1,0,0,1,0,0,3.0
26,2023-24,0022301230,1610612747,1610612740,LAL,NOP,1630700,Dyson Daniels,NOP,652,...,69.333114,80.882353,28.19,0,1,0,1,0,0,2.0


In [32]:
clutch_games = clutch_games_base.loc[
    (clutch_games_base["action_rank_asc"]==1)
].rename(columns={
    "scoreHome": "scoreHome_at5m",
    "scoreAway": "scoreAway_at5m"
}).merge(
    clutch_games_base_end.loc[
        clutch_games_base_end["action_rank_desc"]==1,
        ["game_id", "scoreHome", "scoreAway"]
    ].rename(columns={
        "scoreHome": "scoreHome_at0m",
        "scoreAway": "scoreAway_at0m"
    }),
    on="game_id"
)

In [33]:
clutch_games.assign(
    score_diff_at5m = lambda row: row["scoreHome_at5m"].astype(int) - row["scoreAway_at5m"].astype(int),
    score_diff_at0m = lambda row: row["scoreHome_at0m"].astype(int) - row["scoreAway_at0m"].astype(int)
).pivot_table(
    index="score_diff_at5m",
    columns="score_diff_at0m",
    values="game_id",
    aggfunc="nunique"
).to_clipboard()

In [34]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=back_end_front_end["player_name"],
        y=back_end_front_end["1 of 2 diff from 2 of 2"],
        hovertemplate="<b>%{x}</b><br>"
                      "<b>% Diff</b>: %{y: .1%}"
                      "<extra></extra>",
#         marker_color=["red" if p in ["Andre Drummond", "Mitchell Robinson"] else "blue" for p in ft_data_pivot["player_name"]]
    )
)
fig.update_layout(
    xaxis=dict(
        categoryorder="total ascending",
        tickangle=-45,
        tickfont=dict(size=8)
    ),
    yaxis=dict(tickformat="0%"),
    plot_bgcolor="white",
)
fig.add_hline(
    y=back_front_league_average_diff,
    line=dict(dash="dot", width=1)
)
fig.add_annotation(
    xref="paper",
    x=0.65,
    y=back_front_league_average_diff+(back_front_league_average_diff*.15),
    showarrow=False,
    text=f"<b>Group Average Diff: {round(back_front_league_average_diff*100,1)}%</b>",
    font=dict(size=12)
)

NameError: name 'back_end_front_end' is not defined